In [42]:
import pandas as pd


I part 2 starter vi ud med at indlæse vores 'train_set.csv', 'validation_set.csv' og 'test_set.csv'

In [43]:
train_data = pd.read_csv('train_set.csv')
val_data = pd.read_csv('validation_set.csv')
test_data = pd.read_csv('test_set.csv')

Nu konvertere 'type' kolonnen til en binær klassificering, hvor 1 er lig med reliable news og 0 er lig fake news.

In [51]:
# Definer dine kategorier
reliable_categories = ['reliable', 'political']
fake_categories = ['fake', 'bias', 'conspiracy', 'hate', 'junksci', 'rumor', 'unreliable', 'satire']

# Opret en funktion, der omdanner 'type' til binære værdier
def map_to_binary(article_type):
    if article_type in reliable_categories:
        return 1  # 'reliable'
    elif article_type in fake_categories:
        return 0  # 'fake'
    else:
        return 0  # Sikrer, at funktionen returnerer 0 for alle andre tilfælde, hvilket eliminerer risikoen for NaN-værdier

Her opretter vi vores baseline model ... 

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score


# Indlæs træningsdata
train_data = pd.read_csv('train_set.csv')

# Erstat NaN-værdier i 'stemmed_content' med en tom streng
train_data['stemmed_content'].fillna('', inplace=True)

# Opdater 'type' kolonnen i 'train_data' med de binære værdier
train_data['type'] = train_data['type'].apply(map_to_binary)

# Nu er 'train_data['type']' klar til brug som 'y_train'
y_train = train_data['type']
X_train = train_data['stemmed_content']

# Opret en pipeline med TF-IDF og Naive Bayes
baseline_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', MultinomialNB())
])

# Træn modellen på træningsdata
baseline_pipeline.fit(X_train, y_train)

# Her er den del hvor vi indlæser valideringsdata og forudsiger
# Antager at du har en 'val_set.csv' som valideringsdatasæt
val_data = pd.read_csv('validation_set.csv')
val_data['stemmed_content'].fillna('', inplace=True)

# Opdater 'type' kolonnen i 'val_data' med de binære værdier
val_data['type'] = val_data['type'].apply(map_to_binary)

X_val = val_data['stemmed_content']
y_val = val_data['type']

# Brug pipelinen til at forudsige på valideringsdata
y_pred = baseline_pipeline.predict(X_val)

# Beregn og udskriv præcision for valideringsdata
val_accuracy = accuracy_score(y_val, y_pred)
print(f"Validation accuracy: {val_accuracy}")
f1 = f1_score(y_val, y_pred)
print(f"F1-Score: {f1:.4f}")


Validation accuracy: 0.8295879396984924
F1-Score: 0.7692


# Task 2

Iforhold til meta data overvejede vi domain men i og med vi tidligere fandt ud af at hver domain kun udgiver 1 type artikel mener vi at det vil "overfitte" vores model, hvor vores model bare genkender domain istedet for at forudsige. Vi valgte i stedet at prøve med authors, hvor det viser sig at accuracy stiger med ca. 10%, med author som meta-feature rammer vi nu en accuracy på 92,2 %. 

In [46]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Indlæs træningsdata
train_data = pd.read_csv('train_set.csv')
train_data['stemmed_content'].fillna('', inplace=True)
train_data['authors'].fillna('', inplace=True)

# Indlæs valideringsdata
val_data = pd.read_csv('validation_set.csv')
val_data['stemmed_content'].fillna('', inplace=True)
val_data['authors'].fillna('', inplace=True)

# Omdan 'type' til binære værdier
train_data['type'] = train_data['type'].apply(map_to_binary)
val_data['type'] = val_data['type'].apply(map_to_binary)

# Forbered features og labels
X_train = train_data[['stemmed_content', 'authors']]
y_train = train_data['type']
X_val = val_data[['stemmed_content', 'authors']]
y_val = val_data['type']

# Opret en ColumnTransformer til at forarbejde de forskellige kolonner
preprocessor = ColumnTransformer(
    transformers=[
        ('text', TfidfVectorizer(), 'stemmed_content'),
        ('authors', OneHotEncoder(handle_unknown='ignore'), ['authors'])
    ]
)

# Opret en pipeline med preprocessor og Naive Bayes
baseline_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', MultinomialNB())
])

# Træn modellen på træningsdata
baseline_pipeline.fit(X_train, y_train)

# Forudsige på valideringsdata
y_pred = baseline_pipeline.predict(X_val)

# Beregn og udskriv præcision for valideringsdata
val_accuracy = accuracy_score(y_val, y_pred)
print(f"Validation accuracy: {val_accuracy}")

f1 = f1_score(y_val, y_pred)
print(f"F1-Score: {f1:.4f}")

Validation accuracy: 0.9213668341708543
F1-Score: 0.8985


# task 3

Indlæser filen fra exercise 2 - scraped_articles.csv

In [27]:
scraped_articles = pd.read_csv('scraped_articles.csv')

Da disse artikler ikke er blevet renset eller lavet stemming på endnu, vil vi starte med at gøre dette:

In [28]:
import re

def clean_text(text):
    if pd.isna(text):  # Tjekker for NaN værdier og konverterer dem til en tom streng
        return ''
    text = str(text).lower()  # Sikrer at tekst er en streng og konverterer til små bogstaver
    text = re.sub(r'\s+', ' ', text)  # Erstatter al hvidplads med et enkelt mellemrum
    text = re.sub(r'\b\d{4}-\d{2}-\d{2}\b', '<DATE>', text)  # Erstatter datoer med pladsholder
    text = re.sub(r'\d+', '<NUM>', text)  # Erstatter tal med pladsholder
    text = re.sub(r'\S+@\S+', '<EMAIL>', text)  # Erstatter e-mails med pladsholder
    text = re.sub(r'https?://\S+', '<URL>', text)  # Erstatter URL'er med pladsholder
    text = re.sub(r'[^\w\s]', '', text)  # Fjerner tegnsætning
    return text

# Path to the scraped articles CSV file
csv_file_path = 'scraped_articles.csv'  # Erstat med din faktiske filsti

# Load the scraped articles data from the CSV file
scraped_articles = pd.read_csv(csv_file_path, dtype=str)

# Apply the cleaning function to the 'text' column which contains the article text
scraped_articles['cleaned_text'] = scraped_articles['text'].apply(clean_text)

# Keeping only the 'cleaned_text' column for further processing
cleaned_scraped_articles = scraped_articles[['cleaned_text']]

# Optional: Save the cleaned data to a new CSV file if needed
# cleaned_scraped_articles.to_csv('path_to_your_file/cleaned_scraped_articles.csv', index=False)
cleaned_scraped_articles.head


<bound method NDFrame.head of                                            cleaned_text
0     we know whats coming east ukraine braces for r...
1     in eastern ukraine the tide of this war hasnt ...
2     in order to preserve life and encirclement i h...
3     fighting has been raging in ukraine for two ye...
4     deadly explosions have rocked ukraines souther...
...                                                 ...
5762  indias capital delhi has banned motorbike taxi...
5763  two years ago indian prime minister narendra m...
5764  the discovery of two charred bodies in a burnt...
5765  the judges of masterchef india have been criti...
5766  ruchelle barrie remembers feeling out of place...

[5767 rows x 1 columns]>

In [29]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Download stopwords from NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Load stop words
stop_words = set(stopwords.words('english'))

# Initialize the stemmer
stemmer = PorterStemmer()

def remove_stopwords_and_stem(text):
    # Tokenize the text into words
    words = word_tokenize(text)
    # Remove stop words and stem each word
    return ' '.join([stemmer.stem(word) for word in words if word not in stop_words])

# Apply the function to remove stop words and stem the words in the 'cleaned_text' column
# Brug .loc til at undgå SettingWithCopyWarning
cleaned_scraped_articles.loc[:, 'stemmed_content'] = cleaned_scraped_articles['cleaned_text'].apply(remove_stopwords_and_stem)

# Vis de første par rækker for at bekræfte ændringerne
cleaned_scraped_articles.head()



[nltk_data] Downloading package punkt to /Users/adamjohn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adamjohn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/var/folders/kl/5qdj5c_s0fs5_ym1bdzn3k1c0000gn/T/ipykernel_56807/1439455173.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_scraped_articles.loc[:, 'stemmed_content'] = cleaned_scraped_articles['cleaned_text'].apply(remove_stopwords_and_stem)


,cleaned_text,stemmed_content
0,we know whats coming east ukraine braces for r...,know what come east ukrain brace russian advan...
1,in eastern ukraine the tide of this war hasnt ...,eastern ukrain tide war hasnt chang come fast ...
2,in order to preserve life and encirclement i h...,order preserv life encircl withdrawn unit avdi...
3,fighting has been raging in ukraine for two ye...,fight rage ukrain two year sinc russia invas m...
4,deadly explosions have rocked ukraines souther...,deadli explos rock ukrain southern port citi o...


Nu har vi altså renset og stemmed vores scraped_articles.csv og lavet en ny kolonne i dataen, som hedder cleaned_stemmed_text. Det næste vi vil gøre er at oprette en kolonne 'type' som fortæller om artiklen er reliable eller fake. Her vil vi vurdere alle artiklerne til at være reliable, da de kommer fra bbc. Dette vil altså sige at der kommer til at stå et 1 til under 'type' for alle artiklerne, da det betyder reliable i vores binære klassificering. 

In [30]:
cleaned_scraped_articles['type'] = 1

cleaned_scraped_articles.to_csv('cleaned_scraped_articled')

cleaned_scraped_articles



/var/folders/kl/5qdj5c_s0fs5_ym1bdzn3k1c0000gn/T/ipykernel_56807/2909232008.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_scraped_articles['type'] = 1


,cleaned_text,stemmed_content,type
0,we know whats coming east ukraine braces for r...,know what come east ukrain brace russian advan...,1
1,in eastern ukraine the tide of this war hasnt ...,eastern ukrain tide war hasnt chang come fast ...,1
2,in order to preserve life and encirclement i h...,order preserv life encircl withdrawn unit avdi...,1
3,fighting has been raging in ukraine for two ye...,fight rage ukrain two year sinc russia invas m...,1
4,deadly explosions have rocked ukraines souther...,deadli explos rock ukrain southern port citi o...,1
...,...,...,...
5762,indias capital delhi has banned motorbike taxi...,india capit delhi ban motorbik taxi road deal ...,1
5763,two years ago indian prime minister narendra m...,two year ago indian prime minist narendra modi...,1
5764,the discovery of two charred bodies in a burnt...,discoveri two char bodi burnt vehicl india har...,1
5765,the judges of masterchef india have been criti...,judg masterchef india criticis social media le...,1


Nu hvor vi har en kolonne med stemmed_content hvor alle er sat til reliable under type, vil vi sammensætte vores train_data og cleaned_scraped_articles. 

In [31]:
train_scraped_combined = pd.concat([train_data, cleaned_scraped_articles], ignore_index=True)

# Vis de første par rækker af den nye kombinerede DataFrame for at bekræfte, at alt ser korrekt ud
train_scraped_combined.head()

,Unnamed: 0,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source,tokens,stemmed_content,cleaned_text
0,NUM,NUM,pravdareportcom,0,URL,russia launches major upgrade of air defence s...,DATE NUMNUMNUMNUM,DATE NUMNUMNUMNUM,DATE NUMNUMNUMNUM,russia launches major upgrade of air defence s...,dmitry sudakov lyuba lulko oleg artyukov,NaN,russian air defense antiaircraft systems colle...,first and foremost it goes about antiaircraft ...,missile defense sNUM antiaircraft russian weap...,NaN,NaN,russia launches major upgrade air defence syst...,russia launch major upgrad air defenc system m...,NaN
1,NUM,NUM,nytimescom,1,URL,washington house republicans including newly ...,DATE NUMNUMNUMNUM,DATE NUMNUMNUMNUM,DATE NUMNUMNUMNUM,house gop to vote next week on earmarks ban,david m herszenhorn,NaN,united states politics and government earmarks...,republican leaders called on president obama t...,times topic earmarks,NaN,nytimes,washington house republicans including newly e...,washington hous republican includ newli elect ...,NaN
2,NUM,NUM,thepostemailcom,0,URL,freedoms for which so many have died to defend...,NUMNUMNUMtNUMNUMNUMNUM,DATE NUMNUMNUMNUM,DATE NUMNUMNUMNUM,kim il sung archives,jeffrey harrison jonathan david mooers,NaN,NaN,NaN,NaN,NaN,NaN,freedoms many died defend eroded eyes sharon r...,freedom mani die defend erod eye sharon rondea...,NaN
3,NUM,NUM,nytimescom,1,URL,photo shuffling one foot at a time with the he...,DATE NUMNUMNUMNUM,DATE NUMNUMNUMNUM,DATE NUMNUMNUMNUM,jury selection begins in fraud trial of brooke...,john eligon,NaN,astor brooke marshall anthony d frauds and swi...,anthony marshall has been charged with defraud...,NaN,NaN,nytimes,photo shuffling one foot time help oak cane so...,photo shuffl one foot time help oak cane son b...,NaN
4,NUM,NUM,thenewsdoctorscom,0,URL,former reagan administration white house budge...,NUMNUMNUMtNUMNUMNUMNUM,DATE NUMNUMNUMNUM,DATE NUMNUMNUMNUM,trump faces debt burden madness david stockman...,guest post,NaN,finacial gold news silver news precious metals...,NaN,dave kranzler alan caruba wall street for main...,NaN,NaN,former reagan administration white house budge...,former reagan administr white hous budget dire...,NaN


In [32]:
train_scraped_combined.to_csv('train_scraped_combined.csv', index=False)


Nu prøver vi at bruge vores simple baseline "naive bayes" og ser om de 5767 ekstra artikler kan øge præcisionen.

In [40]:
# Definerer pipeline
baseline_pipeline = pipeline(
    TfidfVectorizer(),
    MultinomialNB()
)

# Træn modellen på den nye kombinerede træningsdata
baseline_pipeline.fit(train_scraped_combined['stemmed_content'], train_scraped_combined['type'])

# Antag at 'X_val' og 'y_val' er defineret og forberedt korrekt fra din valideringsdatasæt

# Brug pipeline til at forudsige på valideringsdata
y_pred = baseline_pipeline.predict(X_val)

# Beregn og udskriv præcision for valideringsdata
val_accuracy = accuracy_score(y_val, y_pred)
print(f"Validation accuracy with additional reliable data: {val_accuracy}")

f1 = f1_score(y_val, y_pred)
print(f"F1-Score: {f1:.4f}")

Validation accuracy: 0.5863417085427136
F1-Score: 0.4334


Som vi kan se på resultatet er nøjagtigheden blevet øget med ca. 0,3% hvilket giver god mening efter som vi har tilføjet 5767 artikler til train_data som i forvejen bestod af 796000 artikler. Da det kun er 5767 artikler vi har tilføjet til train_data havde vi også forventet en meget lille forbedring i nøjagtigheden. Vi har altså kun øget træningssættet med 0,72% og dermed skulle det ikke forbedre nøjagtigheden vildt meget. 

Part 3

Dette er vores advanced moded, efter nogle forsøg med forskellige metoder bla. under skabelsen af baseline modellen, kom vi frem til at logistic regression viste os de bedste resultater. Ligesom i baselinemodellen viste tifløjelse af domain et tegn på overfitting hvor vi nu fik 1.0 validation accuracy. Derfor tilføjer vi igen authors som en meta feature og kan se at modellen giver os en højere score gående fra fra 

In [49]:
print(train_data['type'].value_counts())


type
0    465516
1    330484
Name: count, dtype: int64


In [50]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

# Antager at train_data og val_data allerede er indlæst

# Erstat NaN-værdier i 'stemmed_content' og 'domain' med en tom streng
train_data['stemmed_content'].fillna('', inplace=True)
train_data['authors'].fillna('', inplace=True)

val_data['stemmed_content'].fillna('', inplace=True)
val_data['authors'].fillna('', inplace=True)

# Anvend denne funktion på dit datasæt for at oprette en binær label
y_train = train_data['type'].apply(map_to_binary)
y_val = val_data['type'].apply(map_to_binary)

# Forbered features
X_train = train_data[['stemmed_content', 'authors']]
X_val = val_data[['stemmed_content', 'authors']]

# Opret en ColumnTransformer til at forarbejde de forskellige kolonner
preprocessor = ColumnTransformer(
    transformers=[
        ('text', TfidfVectorizer(), 'stemmed_content'),
        ('authors', OneHotEncoder(handle_unknown='ignore'), ['authors'])
    ]
)

# Opret en pipeline med preprocessor og logistisk regression
advanced_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

# Træn modellen på træningsdata
advanced_pipeline.fit(X_train, y_train)

# Forudsige på valideringsdata
y_pred = advanced_pipeline.predict(X_val)

# Beregn og udskriv præcision for valideringsdata
val_accuracy = accuracy_score(y_val, y_pred)
print(f"Validation accuracy: {val_accuracy}")

# Beregn MSE ved at sammenligne forudsagte sandsynligheder med de faktiske labels
y_pred_probs = advanced_pipeline.predict_proba(X_val)[:, 1]
mse_logistic = mean_squared_error(y_val, y_pred_probs)
print(f"LogisticRegression MSE: {mse_logistic}")
f1 = f1_score(y_val, y_pred)
print(f"F1-Score: {f1:.4f}")

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0

Part 4

Her bruger vi så vores baseline og advanced pipelines til at evaluaere på test settet vi opdelte i Part 1. 

In [ ]:

test_data = pd.read_csv('test_set.csv')
test_data['stemmed_content'].fillna('', inplace=True)
test_data['type'] = test_data['type'].apply(map_to_binary)  # Brug din funktion til at mappe til binære værdier

# Forbered test features og labels
X_test = test_data[['stemmed_content', 'authors']]  # Brug liste notation for at tilgå flere kolonner
y_test = test_data['type']


# Brug din pipeline til at forudsige på test sættet
y_pred_test = baseline_pipeline.predict(X_test)

# Beregn og udskriv nøjagtighed og F1-score for test sættet
test_accuracy = accuracy_score(y_test, y_pred_test)
print(f"Test accuracy: {test_accuracy}")
f1 = f1_score(y_test, y_pred_test,)  # Brug 'weighted' hvis klasserne er ubalancerede
print(f"F1-score: {f1}")

Test accuracy: 0.923175879396985
F1-score: 0.904659748553183


In [ ]:
# Forbered test features og labels
X_test = test_data[['stemmed_content', 'authors']]  # Brug liste notation for at tilgå flere kolonner
y_test = test_data['type']


# Brug din pipeline til at forudsige på test sættet
y_pred_test = advanced_pipeline.predict(X_test)

# Beregn og udskriv nøjagtighed og F1-score for test sættet
test_accuracy = accuracy_score(y_test, y_pred_test)
print(f"Test accuracy: {test_accuracy}")
f1 = f1_score(y_test, y_pred_test,)  # Brug 'weighted' hvis klasserne er ubalancerede
print(f"F1-score: {f1}")

Test accuracy: 0.9380703517587939
F1-score: 0.925234778810454
